In [1]:
import shap
import joblib
import sys,os,time,platform

if platform.node() == 'aizat-fyp':
    sys.path.append(r'/home/aizat/ember')
    sys.path.append(r'/home/aizat/OneDrive/Master Project/Workspace/boosting-ember')
    datasetpath = '/home/aizat/OneDrive/Master Project/Workspace/dataset/ember2018'
else:
    sys.path.append(r'/root/ember')
    sys.path.append(r'/root/boosting-ember')
    datasetpath = '/root/dataset/ember2018/'
import mlflow    
import ember
import pandas as pd
import numpy as np
import lightgbm as lgb
import boostember
from boostember.features_extended import *

boostember.mlflowsetup('/root/boosting-ember/mlflow')

True

In [2]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features(datasetpath)
delunlabel = (y_train != -1)

X_train = X_train[delunlabel]
#X_train = pd.DataFrame(X_train, columns=emberfeaturesheader())
y_train = y_train[delunlabel]
X_train.shape, y_train.shape

((600000, 2381), (600000,))

In [3]:
mlflow.set_tracking_uri("https://atlascompanion.live/")
#mlflow.create_experiment("", artifact_location="gs://mlflow_aizat")
mlflow.set_experiment("Demo")

In [4]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

In [5]:
from mlflow.tracking import MlflowClient
import mlflow.pyfunc

client = MlflowClient()
client.list_artifacts("e82bce1a1b6542b88c982ef9302e7daf")
client.download_artifacts("e82bce1a1b6542b88c982ef9302e7daf", "best_estimator/model.pkl", "model")

model = joblib.load('./model/best_estimator/model.pkl')
#model = mlflow.pyfunc.load_model(model_uri=f"models:/Demo/1")


In [11]:
from cuml.explainer import KernelExplainer

cu_explainer = KernelExplainer(model=model.predict, data=X_train)

MemoryError: std::bad_alloc: CUDA error at: /root/anaconda3/envs/fyp/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [10]:
cu_shap_values = cu_explainer.shap_values(X_test)

Using synchronous transfer as pinned memory (1904800000 bytes) could not be allocated. This generally occurs because of insufficient host memory. The original error was: cudaErrorMemoryAllocation: out of memory


MemoryError: std::bad_alloc: CUDA error at: /root/anaconda3/envs/fyp/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [ ]:
cu_shap_values